# Tool Streaming

Real-time progress updates from streaming tools with async generators

## Imports

In [ ]:
from datetime import datetime
import asyncio
from strands import Agent, tool
from strands.models.openai import OpenAIModel

## Setup Local LLM Client

In [ ]:
model = OpenAIModel(
    client_args={
        "api_key": "none",
        "base_url": "http://localhost:1234/v1",
    },
    model_id="local-model",
    params={
        "temperature": 0.1,
        "top_p": 0.95,
        "frequency_penalty": 0.0,
        "presence_penalty": 0.0,
    }
)

## Streaming Tool

In [ ]:
@tool
async def process_dataset(records: int) -> str:
    """Process records with progress updates."""
    start = datetime.now()

    for i in range(records):
        await asyncio.sleep(0.1)
        if i % 10 == 0:
            elapsed = datetime.now() - start
            yield f"Processed {i}/{records} records in {elapsed.total_seconds():.1f}s"

    yield f"Completed {records} records in {(datetime.now() - start).total_seconds():.1f}s"

## Create Agent

In [ ]:
agent = Agent(
    model=model,
    tools=[process_dataset],
    system_prompt="You are a helpful assistant."
)

## Tool Streaming Function

In [ ]:
async def tool_stream_example():
    async for event in agent.stream_async("Process 50 records"):
        if tool_stream := event.get("tool_stream_event"):
            if update := tool_stream.get("data"):
                print(f"Progress: {update}")
        elif "data" in event:
            print(event["data"], end="", flush=True)

## Run Agent

In [ ]:
await tool_stream_example()